In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


def parse_trades(url):
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to load page {url}")
    
    soup = BeautifulSoup(response.content, 'html.parser')
    trades = []
    
    # Assuming trades are in a table, you might need to adjust the selectors based on the actual HTML structure
    table = soup.find('table')
    if not table:
        raise Exception("No table found on the page")
    
    headers = [header.text for header in table.find_all('th')]
    rows = table.find_all('tr')[1:]  # Skip the header row

    for row in rows:
        cells = row.find_all('td')
        if len(cells) == len(headers):  # Ensure the row has the correct number of cells
            trade = {headers[i]: cells[i].text.strip() for i in range(len(cells))}
            trades.append(trade)
    
    print(f"Number of rows parsed: {len(trades)}")  # Debugging statement
    
    return trades


url = "https://www.capitoltrades.com/trades?politician=P000197&pageSize=96"
trades = parse_trades(url)
trades = pd.DataFrame(trades)
trades.head()

Number of rows parsed: 56


,Politician,Traded Issuer,Published,Traded,Filed After,Owner,Type,Size,Price,
0,Nancy PelosiDemocratHouseCA,Alphabet IncGOOGL:US,20 Jan2025,14 Jan2025,days3,Spouse,buy,250K–500K,N/A,Goto trade detail page.
1,Nancy PelosiDemocratHouseCA,Amazon.com IncAMZN:US,20 Jan2025,14 Jan2025,days3,Spouse,buy,250K–500K,N/A,Goto trade detail page.
2,Nancy PelosiDemocratHouseCA,Apple IncAAPL:US,20 Jan2025,31 Dec2024,days17,Spouse,sell,5M–25M,$250.42,Goto trade detail page.
3,Nancy PelosiDemocratHouseCA,NVIDIA CorporationNVDA:US,20 Jan2025,31 Dec2024,days17,Spouse,sell,1M–5M,$134.29,Goto trade detail page.
4,Nancy PelosiDemocratHouseCA,NVIDIA CorporationNVDA:US,20 Jan2025,20 Dec2024,days28,Spouse,buy,500K–1M,$134.70,Goto trade detail page.


In [24]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# List of companies
companies = ['GOOGL', 'AMZN', 'AAPL', 'NVDA', 'PANW', 'TEM', 'VST', 'MSFT', 'AVGO', 'TSLA', 'V', 'RBLX', 'AB', 'NFLX', 'PYPL', 'CRM', 'DIS', 'MU', 'WBD', 'T']

# Define the time period
end_date = datetime.now()
start_date = end_date - timedelta(days=3*365)

# Fetch data for each company
all_data = pd.DataFrame()

for company in companies:
    ticker = yf.Ticker(company)
    data = ticker.history(start=start_date, end=end_date)
    data['Company'] = company  # Add a column for the company name
    all_data = pd.concat([all_data, data])

print("Data collection complete.")
all_data.head()

Data collection complete.


,Open,High,Low,Close,Volume,Dividends,Stock Splits,Company
Date,,,,,,,,
2022-02-22 00:00:00-05:00,129.369022,131.703059,127.760351,129.344620,44990000,0.0,0.0,GOOGL
2022-02-23 00:00:00-05:00,131.172986,131.574029,127.066869,127.127151,32786000,0.0,0.0,GOOGL
2022-02-24 00:00:00-05:00,124.502168,132.519605,124.502168,132.211716,54456000,0.0,0.0,GOOGL
2022-02-25 00:00:00-05:00,133.069608,134.782895,131.275605,133.973831,36406000,0.0,0.0,GOOGL
2022-02-28 00:00:00-05:00,132.569429,135.261163,132.113577,134.569183,38864000,0.0,0.0,GOOGL


ValueError: time data "12 Sept2024" doesn't match format "%d %b%Y", at position 1. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.